# Huggingface 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
!pip install evaluate
import evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

from datasets import load_dataset, Dataset
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from transformers import AutoTokenizer, DataCollatorWithPadding, TFAutoModelForSequenceClassification

pd.options.display.max_colwidth = 0

In [4]:
train_data = pd.read_csv('/kaggle/input/nlp-getting-started/train.csv')
train_data['keyword'].fillna("none", inplace=True)
train_data['location'].fillna("none", inplace=True)
train_data.sample(n=10, random_state=23)

,id,keyword,location,text,target
1461,2109,catastrophe,none,.@robdelaney Catastrophe is anything but! I literally have been unable to stop ejaculating.,1
1837,2642,crashed,too far,He was only .4 of a second faster than me and I overtook him twice (then crashed) tru luv &lt;3 &lt;3,0
6144,8763,siren,none,Internet firms to be subject to new cybersecurity rules in EU... http://t.co/lafTJ2GyLY,0
2354,3389,demolition,"ÌÏT: 36.142163,-95.979189",@samajp32 really needs to tone it down some in the weight room. RT @SoonerSportsTV: Demolition (cont) http://t.co/2o7Eva1cOe,0
4765,6781,lightning,Elchilicitanierraversal,#NowPlaying 'The Lightning Strike' de Snow Patrol de A Hundred Million Suns ? http://t.co/GrzcHkDF37,0
6882,9866,traumatised,cork,@AnnmarieRonan @niamhosullivanx I can't watch tat show its like a horror movie to me I get flashbacks an everything #traumatised,0
3566,5097,famine,none,when things become terrible more than the great Ethiopian famine...,0
1847,2657,crush,none,'@jorrynja: 6. @ your bf/gf/crush ??' @Ter_ell ??,1
4961,7071,meltdown,"Colorado, USA",@nprfreshair I really can't believe he is skipping out before the Republican meltdown...I mean 'debate'.,0
7021,10064,typhoon,"Ibadan,Oyo state",Obama Declares Disaster for Typhoon-Devastated Saipan: Obama signs disaster declaration for Northern Marians a... http://t.co/UsVyHdG9OG,1


## Process Tweets

In [5]:
!pip install contractions

import contractions, emoji, re, unidecode

from nltk.stem import PorterStemmer

class TweetSweeper:
    def __init__(self, tweets):
        self.tweets = tweets
        self.stemmer = PorterStemmer()
        self.tweets['hashtags'] = self.tweets['text'].apply(lambda x: [x for x in x.split(" ") if x.startswith("#")])
        # Keyword
        self.tweets['keyword'] = self.tweets['keyword'].apply(str)
        self.tweets['keyword'] = self.tweets['keyword'].apply(lambda x: self.clean_keywords(x))
        self.tweets['keyword'] = self.tweets['keyword'].apply(lambda x: self.remove_accents(x))
        self.tweets['keyword'] = self.tweets['keyword'].apply(lambda x: self.remove_punctuation(x))
        self.tweets['keyword'].fillna("none", inplace=True)
        self.tweets['stems'] = self.tweets['keyword'].apply(lambda x: self.stemmer.stem(x))
        # Location
        self.tweets['location'] = self.tweets['location'].apply(str)
        self.tweets['location'] = self.tweets['location'].apply(lambda x: self.remove_accents(x))
        self.tweets['location'] = self.tweets['location'].apply(lambda x: self.remove_punctuation(x))
        self.tweets['location'] = self.tweets['location'].apply(lambda x: self.remove_extra_w_space(x))
        self.tweets['location'] = self.tweets['location'].apply(lambda x: self.remove_nums(x))
        # Counts
        self.tweets['emojis'] = self.tweets['text'].apply(self.emoji_count)
        self.tweets['num_hashtags'] = self.tweets['text'].apply(self.hash_count)
        self.tweets['token_count'] = self.tweets['text'].apply(lambda x: len(x))
        # Tweet Text
        self.tweets['text_clean'] = self.tweets['text'].apply(lambda x: contractions.fix(x))
        self.tweets['text_clean'] = self.tweets['text_clean'].apply(lambda x: self.demojize(x))
        self.tweets['text_clean'] = self.tweets['text_clean'].apply(lambda x: self.tweet_clean(x))
        # Text Prompt
        self.tweets['text_prompt'] = self.tweets.apply(
            lambda row: " [SEP] tweet: " + row['text_clean'] + 
                        " [SEP] keyword: " + row['keyword'] +
                        " [SEP] location: " + row['location'] +
                        " [SEP] token count: " + str(row['token_count']), axis=1)        
        # TF processing
        # ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).
        self.tweets['keyword'] = self.tweets['keyword'].apply(str)
        self.tweets['location'] = self.tweets['location'].apply(str)
        # Failed to convert a NumPy array to a Tensor (Unsupported object type list).
        self.tweets['hashtags'] = self.tweets['hashtags'].apply(str)
        
    def tweet_clean(self, text):
        text = text.encode('ascii', 'ignore').decode()  # remove unicode characters
        text = re.sub(r'https*\S+', ' ', text) # remove links
        text = re.sub(r'http*\S+', ' ', text)
        text = text.replace("%20", " " )
        text = re.sub(r'\'\w+', '', text)
        text = re.sub(r'\w*\d+\w*', '', text)
        text = re.sub(r'\s{2,}', ' ', text)
        text = re.sub(r'\s[^\w\s]\s', '', text)
        # Remove '@name'
        text = re.sub(r'(@.*?)[\s]', ' ', text)
        # Replace '&amp;' with '&'
        text = re.sub(r'&amp;', '&', text)
        # Remove trailing whitespace
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def demojize(self, tweet):
        emojis = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            "]+", flags = re.UNICODE)
        cleaned_text = emojis.sub(r'', tweet)
        return cleaned_text
    
    def emoji_count(self, tweet):
        tweet = emoji.demojize(tweet, delimiters=('__','__'))
        pattern = r'_+[a-z_&]+_+'
        return len(re.findall(pattern, tweet))
    
    def hash_count(self, string):
        words = string.split()
        hashtags = [word for word in words if word.startswith('#')]
        return len(hashtags)
    
    def clean_keywords(self, keyword):
        cleaned = re.sub(r'%20', ' ', keyword)
        return cleaned
    
    def remove_accents(self, keyword):
        cleaned = unidecode.unidecode(keyword)
        return cleaned
    
    def remove_punctuation(self, keyword):
        cleaned = re.sub(r"[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n -' ]"," ",keyword)
        return cleaned
    
    def remove_nums(self, location):
        cleaned = re.sub(r'\d+', '', location)
        return cleaned

    def remove_extra_w_space(self, location):
        cleaned_text = re.sub(r"\s+"," ", location).strip()
        return cleaned_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 12.5 MB/s eta 0:00:00


In [6]:
sweep_data = TweetSweeper(train_data).tweets.copy()
sweep_data.sample(5)

,id,keyword,location,text,target,hashtags,stems,emojis,num_hashtags,token_count,text_clean,text_prompt
4428,6301,hostage,Indiana,Who is Tomislav Salopek the Islamic State's Most Recent Hostage? - http://t.co/wiQJERUktF,0,[],hostag,0,0,89,Who is Tomislav Salopek the Islamic State Most Recent Hostage?,[SEP] tweet: Who is Tomislav Salopek the Islamic State Most Recent Hostage? [SEP] keyword: hostage [SEP] location: Indiana [SEP] token count: 89
7110,10187,violent storm,Costa Rica,RT '@NASASolarSystem: Jupiter's Red Spot is a violent storm larger than the entire Earth: http://t.co/i0Tvl15CoZ http://t.co/IgtXhapO0K,0,[],violent storm,0,0,135,RT ' Jupiter Red Spot is a violent storm larger than the entire Earth:,[SEP] tweet: RT ' Jupiter Red Spot is a violent storm larger than the entire Earth: [SEP] keyword: violent storm [SEP] location: Costa Rica [SEP] token count: 135
7193,10305,weapon,Central Florida,BREAKING: Obama Officials GAVE Muslim Terrorist the Weapon Used in Texas Attack http://t.co/RJcaxjp4oS,1,[],weapon,0,0,102,BREAKING: Obama Officials GAVE Muslim Terrorist the Weapon Used in Texas Attack,[SEP] tweet: BREAKING: Obama Officials GAVE Muslim Terrorist the Weapon Used in Texas Attack [SEP] keyword: weapon [SEP] location: Central Florida [SEP] token count: 102
1980,2851,cyclone,none,#Rohingya houses in #Kyee NockThie hamlet from Taungbazar region in Buthidaung were severely damaged in Cyclone @KasitaRoch @VivianUNHCR,1,"['#Rohingya', '#Kyee']",cyclon,0,2,136,#Rohingya houses in #Kyee NockThie hamlet from Taungbazar region in Buthidaung were severely damaged in Cyclone @VivianUNHCR,[SEP] tweet: #Rohingya houses in #Kyee NockThie hamlet from Taungbazar region in Buthidaung were severely damaged in Cyclone @VivianUNHCR [SEP] keyword: cyclone [SEP] location: none [SEP] token count: 136
3100,4450,electrocuted,none,'Hey bitch blow me' uh no. Stick your dick in some water then an outlet so u get electrocuted..,0,[],electrocut,0,0,95,bitch blow me' uh no. Stick your dick in some water then an outlet so you get electrocuted..,[SEP] tweet: bitch blow me' uh no. Stick your dick in some water then an outlet so you get electrocuted.. [SEP] keyword: electrocuted [SEP] location: none [SEP] token count: 95


In [7]:
# Do we really see difference?
sweep_data[['text', 'text_clean']][sweep_data['text'] != sweep_data['text_clean']].sample(10)

,text,text_clean
3643,There have been zero spider bite-related fatalities in Australia since 1979.,There have been zero spider bite-related fatalities in Australia since .
642,Do you know anyone looking to move to Hammond OR? Share this listing! http://t.co/3xn1soh4Bb,Do you know anyone looking to move to Hammond OR? Share this listing!
6675,@KristyLeeMusic brings her Alabama thunder back to the Attic SEPTEMBER 26! Tickets: https://t.co/B7ZwEVsrGO,brings her Alabama thunder back to the Attic SEPTEMBERTickets:
4904,@gigagil IOF murdered over 513 Palestinian children (70% under 12) during Gaza Massacre where was zionist morality-zionism is a world evil!,IOF murdered over Palestinian children (% underduring Gaza Massacre where was zionist morality-zionism is a world evil!
4462,#hot C-130 specially modified to land in a stadium and rescue hostages in Iran in 1980 http://t.co/W0EXzAD5Gc #prebreak #best,#hot C- specially modified to land in a stadium and rescue hostages in Iran in #prebreak #best
4725,I lava you ?????? http://t.co/aeZ3aK1lRN,I lava you ??????
5638,Newlyweds feed thousands of Syrian refugees instead of hosting a banquet wedding dinner http://t.co/EGcv7ybjae #Age #news,Newlyweds feed thousands of Syrian refugees instead of hosting a banquet wedding dinner #Age #news
74,I-77 Mile Marker 31 South Mooresville Iredell Vehicle Accident Ramp Closed at 8/6 1:18 PM,I- Mile Marker South Mooresville Iredell Vehicle Accident Ramp Closed at: PM
5465,Reddit Will Now Quarantine Offensive Content http://t.co/u9BkQt6XHR,Reddit Will Now Quarantine Offensive Content
1065,namjoon's FANTASTIC IS BOMB BYE OMG,namjoon FANTASTIC IS BOMB BYE OMG


## Load Data

In [8]:
# 60:20:20 split
if True:
    df = sweep_data[['id', 'keyword', 'location', 'text_prompt', 'target']].copy()
    df.rename(columns={"text_prompt": "text"}, inplace=True)
    train_ds = Dataset.from_pandas(df)
    val_size= int(0.4 * len(train_ds))
    tv_ds = train_ds.train_test_split(test_size=val_size, seed=42)
else: 
    train_ds = load_dataset('csv', data_files='/kaggle/input/nlp-getting-started/train.csv')
    # load_dataset( .., split="train[:10%]+train[-80%:]"
    val_size= int(0.4 * len(train_ds['train']))
    tv_ds = train_ds['train'].train_test_split(test_size=val_size, seed=42)

test_size = int(val_size/2)
tvt_ds = tv_ds['test'].train_test_split(test_size=test_size, seed=42)
tvt_ds['val'] = tvt_ds['train']
tvt_ds['train'] = tv_ds['train']

print(train_ds)
print(tv_ds)
print(tvt_ds)

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'target'],
    num_rows: 7613
})
DatasetDict({
    train: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target'],
        num_rows: 4568
    })
    test: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target'],
        num_rows: 3045
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target'],
        num_rows: 4568
    })
    test: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target'],
        num_rows: 1522
    })
    val: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'target'],
        num_rows: 1523
    })
})


In [9]:
checkpoint = "bert-base-uncased"
checkpoint = "distilbert-base-uncased"
#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english" # low accuracy on 3 epochs

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

tokenized_ds = tvt_ds.map(tokenize_function, batched=True)

# Transformer models only accept tensors as input: `return_tensors`
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_ds = tokenized_ds['train'].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols=['target'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=15
)

tf_val_ds = tokenized_ds['val'].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols=['target'],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=15
)

tf_test_ds = tokenized_ds['test'].to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'token_type_ids'],
    label_cols=['target'],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=15
)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [10]:
def build_classifier_model(checkpoint, train_ds=tf_train_ds, epochs=3):
    
    batch_size = 8
    # The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
    # by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
    # not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
    num_train_steps = len(tf_train_ds) * epochs
    lr_scheduler = PolynomialDecay(
        initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
    )
    opt = Adam(learning_rate=lr_scheduler)

    model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
    return model

epochs = 30
model = build_classifier_model(checkpoint, epochs)
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.config

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.30.2",
  "vocab_size": 30522
}

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=5)

history = model.fit(tf_train_ds, validation_data=tf_val_ds, epochs=epochs, callbacks=[early_stopping])

Epoch 1/30
304/304 [==============================] - 79s 169ms/step - loss: 0.4797 - accuracy: 0.7879 - val_loss: 0.4171 - val_accuracy: 0.8378
Epoch 2/30
304/304 [==============================] - 27s 90ms/step - loss: 0.3344 - accuracy: 0.8713 - val_loss: 0.3841 - val_accuracy: 0.8431
Epoch 3/30
304/304 [==============================] - 29s 94ms/step - loss: 0.2229 - accuracy: 0.9224 - val_loss: 0.4390 - val_accuracy: 0.8293
Epoch 4/30
304/304 [==============================] - 25s 84ms/step - loss: 0.1751 - accuracy: 0.9417 - val_loss: 0.4390 - val_accuracy: 0.8293
Epoch 5/30
304/304 [==============================] - 27s 90ms/step - loss: 0.1765 - accuracy: 0.9414 - val_loss: 0.4390 - val_accuracy: 0.8293
Epoch 6/30
304/304 [==============================] - 25s 82ms/step - loss: 0.1767 - accuracy: 0.9401 - val_loss: 0.4390 - val_accuracy: 0.8293
Epoch 7/30
304/304 [==============================] - 27s 89ms/step - loss: 0.1759 - accuracy: 0.9414 - val_loss: 0.4390 - val_accuracy

In [13]:
predicted = model.predict(tf_test_ds)
predictions = np.argmax(predicted.logits, axis=1)

metric = evaluate.load('f1')

metric.compute(predictions=predictions, references=tokenized_ds['test']['target'])

102/102 [==============================] - 5s 29ms/step


{'f1': 0.7928068803752931}

## Submission

In [14]:
if True:
    submission_data = pd.read_csv('/kaggle/input/nlp-getting-started/test.csv', index_col='id')

    sweep_submission = TweetSweeper(submission_data).tweets.copy()
    sweep_submission.reset_index(inplace=True)
    df = sweep_submission[['id', 'keyword', 'location', 'text_prompt']].copy()
    df.rename(columns={"text_prompt": "text"}, inplace=True)
    submission_ds = Dataset.from_pandas(df)
else:
    submission_ds = load_dataset('csv', data_files='/kaggle/input/nlp-getting-started/test.csv')

print(submission_ds)

tokenized_submission_ds = submission_ds.map(tokenize_function, batched=True)

tf_submission_ds = tokenized_submission_ds.to_tf_dataset(
    columns=['attention_mask', 'input_ids', 'token_type_ids'],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=1
)

# The output you get is the non-normalized probability for each class (i.e. logits).
# Apply the softmax function to normalize these probabilities.
submission = model.predict(tf_submission_ds)
submission_predicted = np.argmax(submission.logits, axis=1)
#submission_predicted = np.where(tf.nn.softmax(submission.logits).numpy() > 0.5, 1, 0)
submission_predicted

submission_ds = (submission_ds.add_column('target', submission_predicted).remove_columns('keyword location text'.split()))
print(submission_ds)

Dataset({
    features: ['id', 'keyword', 'location', 'text'],
    num_rows: 3263
})


  0%|          | 0/4 [00:00<?, ?ba/s]

3263/3263 [==============================] - 46s 14ms/step
Dataset({
    features: ['id', 'target'],
    num_rows: 3263
})


In [15]:
submission_ds.to_csv('submission.csv', index=False)
!head -10 submission.csv

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
id,target
0,1
2,1
3,1
9,1
11,1
12,1
21,0
22,0
27,0
